# SRCNN Training

### Imports

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

source = None
for root, dirs, files in os.walk(r'/content/drive'):
    for name in dirs:
        if name == 'srcnn':
            source = f"{os.path.abspath(os.path.join(root, name))}{os.sep}implementation.zip"

if source is None: 
  raise FileNotFoundError("Make sure you add a shortcut to Text2Mesh in your drive")
!cp "{source}" "/content/implementation.zip"

!unzip /content/implementation.zip -d .

In [ ]:
import h5py
%matplotlib inline
import matplotlib.image as mpimg
from implementation.test import *
from implementation.train import *
from matplotlib import pyplot as plt
from implementation.srcnn import SRCNN
from implementation.data_utils import create_dataset
from sklearn.model_selection import train_test_split

home_dir = "/content/implementation"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Computation device: ', device)

### Loading Data

In [ ]:
hf_path = create_dataset(f"{home_dir}/inputs/vid1.mp4", home_dir, video=True, max_iters=500)
with h5py.File(hf_path) as file:
    data_inputs = file['data'][:].astype('float32')
    data_labels = file['label'][:].astype('float32') 
    file.close()

train_inputs, val_inputs, train_labels, val_labels = train_test_split(data_inputs, data_labels, test_size=0.25)
print(f"Training samples count:\t{train_inputs.shape[0]}")
print(f"Validation samples count:\t{val_inputs.shape[0]}")

### Learning Parameters

In [ ]:
lr = 0.001 
epochs = 1000
batch_size = 64

### Training & Validation

In [ ]:
model = train_and_validate(device, val_inputs, val_labels, train_inputs, train_labels, batch_size, epochs, lr, home_dir)